# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [30]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [31]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [32]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [33]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [34]:
# Created a dataframe of the csv file to explore the data
file = pd.read_csv('event_datafile_new.csv')
file.head()


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [35]:
#Converted to dataframe to read csv on local system
file.to_csv(index=False)

'artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId\nBarry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile),88\nJimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88\nBuilding 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88\nThe B-52\'s,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38\nDie Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g\'sund,38\nCarrie Underwood,Gianna,F,2,Jones,195.70893,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Look At Me,38\nNo Te Va Gustar,Kaylee,F,0,Summers,246.43873,free,"Phoenix-Mesa-Scottsdale, AZ",833,Ya EntendÃ\x83Â\xad,8\

In [36]:
#Checking the frequency of occurence of individual song in the column
file.song.value_counts(dropna=False)

You're The One                                                         37
Undo                                                                   28
Revelry                                                                27
Sehr kosmisch                                                          21
Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)    19
Secrets                                                                17
Canada                                                                 17
Dog Days Are Over (Radio Edit)                                         16
Invalid                                                                14
ReprÃÂ©sente                                                          14
Fireflies                                                              14
Somebody To Love                                                       13
Home                                                                   13
Yellow                                

In [37]:
#Checking the frequency of occurence of individual userId in the column
file.userId.value_counts(dropna=False)

49     689
80     665
97     557
15     463
44     397
29     346
24     321
73     289
88     270
36     248
16     223
95     213
85     179
30     178
25     169
42     140
58     140
26     114
82      87
72      72
32      56
101     55
50      48
86      45
66      37
37      34
70      33
69      29
10      28
92      27
      ... 
94       7
17       7
28       7
84       6
81       6
4        5
71       5
18       5
7        5
13       5
5        4
98       4
23       4
99       4
68       3
3        3
90       3
38       3
47       3
48       3
64       2
56       2
11       2
22       2
27       2
39       2
59       2
79       2
45       1
87       1
Name: userId, Length: 96, dtype: int64

In [38]:
#Investigating the max value in the userId
file.loc[file["userId"].idxmax()]

artist                       Soul II Soul
firstName                          Jayden
gender                                  M
itemInSession                           0
lastName                              Fox
length                            239.673
level                                free
location         New Orleans-Metairie, LA
sessionId                             298
song                             How Long
userId                                101
Name: 429, dtype: object

In [39]:
#Investigating the min value in the userId
file.loc[file["userId"].idxmin()]

artist            Los Del Rio
firstName             Jizelle
gender                      F
itemInSession               1
lastName             Benjamin
length                249.495
level                    free
location         Plymouth, IN
sessionId                 323
song                 Macarena
userId                      2
Name: 487, dtype: object

# Setting up connection to the Cassandra database
#### Creating a Cluster

In [40]:
# Making a connection to a Cassandra instance on local machine 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [41]:
# Created the KEYSPACE
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [42]:
# Set KEYSPACE to the keyspace udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query for creation of first table -
### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


In [43]:
##Cassandra database supports denormalization in the data modelling.
##Cassandra like other NOSQL databases follows CAP theorem and the tables are modelled based on the queries to be responded to.
# The PRIMARY KEY needs to be unique and it is composed of PARTITION KEY and Clustering column.
# For query 1,query requires to search data based on sessionId and itemInSession hence these are part of the PRIMARY KEY 

query = "CREATE TABLE IF NOT EXISTS music_by_sessionId_and_itemInSession"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
        print(e)

In [44]:
# Code to set up the CSV file and then insert the data into the database 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """INSERT INTO music_by_sessionId_and_itemInSession
            (
                        sessionid,
                        iteminsession,
                        artist,
                        song,
                        length
            ) """
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [45]:
# Check to see if the data was added successfully into the database
query = """SELECT artist,
       song,
       length
FROM   music_by_sessionId_and_itemInSession
WHERE  sessionid = 338
       AND iteminsession = 4"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query for creation of second table -
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [46]:
##Cassandra database supports denormalization in the data modelling.
##Cassandra like other NOSQL databases follows CAP theorem and the tables are modelled based on the queries to be responded to.
# The PRIMARY KEY needs to be unique and it is composed of PARTITION KEY and Clustering column.
# For query 2,query requires to search data based on userId and sessionId hence these are part of the PRIMARY KEY.
# Also the query mentions that the query sort by itemInSession hence that is the clustering column based on which we sort the data for storage within the partitions.

query = "CREATE TABLE IF NOT EXISTS music_sorted_by_itemInSession"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId),itemInSession))"

try:
    session.execute(query)
except Exception as e:
        print(e)                    

In [47]:
# Code to set up the CSV file and then insert the data into the database  
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """ INSERT INTO music_sorted_by_itemInSession
            (
                        userid,
                        sessionid,
                        iteminsession,
                        artist,
                        song,
                        firstname,
                        lastname
            ) """
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [48]:
# Check to see if the data was added successfully into the database
query = """SELECT artist,
       song,
       firstname,
       lastname
FROM   music_sorted_by_itemInSession
WHERE  userid = 10
       AND sessionid = 182"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname,row.lastname)
#       print(row)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Query for creation of third table -
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [49]:
##Cassandra database supports denormalization in the data modelling.
##Cassandra like other NOSQL databases follows CAP theorem and the tables are modelled based on the queries to be responded to.
# The PRIMARY KEY needs to be unique and it is composed of PARTITION KEY and Clustering column.
# For query 3,query requires to search data based on song hence this is part of the PRIMARY KEY.

query = "CREATE TABLE IF NOT EXISTS music_history_sorted_by_song"
query = query + "(song text, firstName text, lastName text, PRIMARY KEY(song))"

try:
    session.execute(query)
except Exception as e:
        print(e) 


In [50]:
# Code to set up the CSV file and then insert the data into the database
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """ INSERT INTO music_history_sorted_by_song
            (
                        song,
                        firstname,
                        lastname
            ) """
        query = query + "VALUES(%s,%s,%s)"
        session.execute(query, (line[9],line[1],line[4]))

In [51]:
# Check to see if the data was added successfully into the database
query = """SELECT firstname,
       lastname
FROM   music_history_sorted_by_song
WHERE  song = 'All Hands Against His Own'"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname,row.lastname)


Jacqueline Lynch


In [52]:
# We get only a single row for the 'All Hands Against His Own' however there are 3 occurences of the same in the event_datafile_new.csv
# Hence we would need to use more than just the song as Primary key.
# userId has values from 2 to 101 hence we can use it for our query in addition to the song column
query = "CREATE TABLE IF NOT EXISTS music_history_sorted_by_song_userId"
query = query + "(song text, userId int,firstName text, lastName text, PRIMARY KEY((song),userId))"

try:
    session.execute(query)
except Exception as e:
        print(e) 


In [53]:
# Code to set up the CSV file and then insert the data into the database
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """ INSERT INTO music_history_sorted_by_song_userId
            (
                        song,
                        userid,
                        firstname,
                        lastname
            ) """
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [54]:
# Check to see if the data was added successfully into the database
query = """SELECT firstname,
       lastname,
       userid,
       song
FROM   music_history_sorted_by_song_userId
WHERE  song = 'All Hands Against His Own'"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname,row.lastname,row.userid,row.song)

Jacqueline Lynch 29 All Hands Against His Own
Tegan Levine 80 All Hands Against His Own
Sara Johnson 95 All Hands Against His Own


### Drop the tables before closing out the sessions

In [55]:
## Drop the table before closing out the sessions

In [56]:
query = "drop table music_by_sessionId_and_itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table music_sorted_by_itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_history_sorted_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    
    
query = "drop table music_history_sorted_by_song_userId"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    
    
    

### Close the session and cluster connection¶

In [57]:
session.shutdown()
cluster.shutdown()